# Quantitative Value Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports
The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

## Importing Our List of Stocks & API Token
As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call
It's now time to make the first version of our value screener!

We'll start by building a simple value screener that ranks securities based on a single metric (the price-to-earnings ratio).

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'symbol': 'AAPL',
 'companyName': 'Apple, Inc.',
 'primaryExchange': 'AQADNS',
 'calculationPrice': 'close',
 'open': 446.3,
 'openTime': 1639687260110,
 'openSource': 'liiafcfo',
 'close': 455.26,
 'closeTime': 1630130836745,
 'closeSource': 'foiaiclf',
 'high': 468.2,
 'highTime': 1675832424355,
 'highSource': 'eya 1edtmuldiirc5 pe ne',
 'low': 456.81,
 'lowTime': 1676076933489,
 'lowSource': ' iela eedndr1puty icm5e',
 'latestPrice': 454.54,
 'latestSource': 'Close',
 'latestTime': 'August 12, 2020',
 'latestUpdate': 1659259644502,
 'latestVolume': 43109172,
 'iexRealtimePrice': None,
 'iexRealtimeSize': None,
 'iexLastUpdated': None,
 'delayedPrice': 462,
 'delayedPriceTime': 1608883270160,
 'oddLotDelayedPrice': 449,
 'oddLotDelayedPriceTime': 1625709713649,
 'extendedPrice': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPriceTime': None,
 'previousClose': 447.9,
 'previousVolume': 48248140,
 'change': 15.04,
 'changePercent': 0.03396,
 'volume': 0,
 'i

## Parsing Our API Call
This API call has the metric we need - the price-to-earnings ratio.

Here is an example of how to parse the metric from our API call:

In [5]:
pe_ratio = data['peRatio']
pe_ratio

34.53

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [8]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,102.49,45.55,N/A
1,AAL,13.84,-1.74,N/A
2,AAP,156.43,28.08,N/A
3,AAPL,473.86,35.37,N/A
4,ABBV,99.76,21.27,N/A
...,...,...,...,...
500,YUM,97.82,28.48,N/A
501,ZBH,142.25,719.96,N/A
502,ZBRA,287.94,31.26,N/A
503,ZION,35.85,13.43,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock". 

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [9]:
final_dataframe.sort_values('Price-to-Earnings Ratio', ascending = False, inplace = True)
final_dataframe = final_dataframe[:51]
final_dataframe.reset_index(drop = True, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,RTX,65.20,2419.87,N/A
1,FIS,145.79,1537.26,N/A
2,GPC,100.29,985.26,N/A
3,ZBH,142.25,719.96,N/A
4,HLT,87.44,602.03,N/A
5,STZ,180.60,590.63,N/A
6,HRB,15.48,561.9,N/A
7,GLW,33.34,257.32,N/A
8,IQV,170.31,247.45,N/A
9,WMB,22.37,212.96,N/A


## Calculating the Number of Shares to Buy
Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

Since we've already done most of the work on this, try to complete the following two code cells without watching me do it first!